### Looking at a tensorflow sampler implementation

This will be probably be a bit too far out. However, the goal is more to show you what is possible rather than showing you how to fully implement with tensorflow. For that, we refer to the *programmer's guide* as a first step that comes with TATi.


Sampling is very similar to training. And training or optimization is done in classes derived from `Optimizer` in Tensorflow. Hence, if we aim to implement a new sampler, then we need to override this class and add our own functionality.

As a start, let us take a look what we have to do when overriding `GradientDescentOptimizer` in `tensorflow`.

In [ ]:
class GradientDescent(tf.train.GradientDescentOptimizer):
    def __init__(self, learning_rate, use_locking=False, name='GradientDescent'):
        self._learning_rate = learning_rate
        super(GradientDescent, self).__init__(learning_rate, use_locking, name)
        
    def _prepare(self):
        self._learning_rate_t = ops.convert_to_tensor(self._learning_rate, name="learning_rate")
        super(GradientDescent, self)._prepare()
        
    def _apply_dense(self, grad, var):
        lr_t = math_ops.cast(self._learning_rate_t, var.dtype.base_dtype)
        scaled_gradient = lr_t * grad
        var_update = state_ops.assign_sub(var, scaled_gradient)
        return control_flow_ops.group(*[var_update])

Then, we step on to the implementation of a sampler, here SGLD.

In [ ]:
class StochasticGradientLangevinDynamicsSampler(tf.train.GradientDescentOptimizer)
    def __init__(self, step_width, inverse_temperature,
                 seed=None, use_locking=False, name='SGLD'):
        super(StochasticGradientLangevinDynamicsSampler, self).__init__(use_locking, name)
        self._step_width = step_width
        self._seed = seed
        self._inverse_temperature = inverse_temperature

    def _prepare(self):
        super(StochasticGradientLangevinDynamicsSampler, self)._prepare()
        self._step_width_t = ops.convert_to_tensor(self._step_width, name="step_width")
        self._inverse_temperature_t = ops.convert_to_tensor(self._inverse_temperature, name="inverse_temperature")

        
    def _prepare_dense(self, grad, var):
        step_width_t = math_ops.cast(self._step_width_t, var.dtype.base_dtype)
        inverse_temperature_t = math_ops.cast(self._inverse_temperature_t, var.dtype.base_dtype)
        if self._seed is None:
            random_noise_t = tf.random_normal(grad.get_shape(), mean=0.,stddev=1., dtype=dds_basetype)
        else:
            # increment such that we use different seed for each random tensor
            self._seed += 1
            random_noise_t = tf.random_normal(grad.get_shape(), mean=0., stddev=1., dtype=dds_basetype, seed=self._seed)
        return step_width_t, inverse_temperature_t, random_noise_t


    def _apply_dense(self, grad, var):
        step_width_t, inverse_temperature_t, random_noise_t = self._prepare_dense(grad, var)

        scaled_gradient = step_width_t * grad

        scaled_noise = tf.sqrt(2.*step_width_t/inverse_temperature_t) * random_noise_t

        var_update = state_ops.assign_sub(var, scaled_gradient + scaled_noise)

        return control_flow_ops.group(*[var_update])
